In [1]:
import os
import numpy

import rospy
from geometry_msgs.msg import WrenchStamped
from realsense2_camera.msg import CustomVector
from realsense2_camera.msg import EstimatedPose

In [2]:
def normalize_rad(angles):
    angles = numpy.array(angles)
    angles = angles % (2*numpy.pi)
    angles = (angles + 2*numpy.pi) % (2*numpy.pi)
    for i in range(len(angles)):
        if (angles[i] > numpy.pi):
            angles[i] -= 2*numpy.pi
    return angles

In [3]:
def ft_callback(data):
    '''ROS Callback function for the force-torque data'''
    global ft_values
    ft_values = numpy.array([data.wrench.force.x, data.wrench.force.y, data.wrench.force.z, data.wrench.torque.x, data.wrench.torque.y, data.wrench.torque.z])  
    #publish_observations(ft_values)

def rotmat_callback(data):
    global x_mat_flat
    x_mat_flat = numpy.array(data.data)

def pose_callback(data):
    '''ROS Callback function for the pose_estimation'''
    global pose
    pose = numpy.array([data.tx, data.ty, data.tz, data.rx, data.ry, data.rz])

In [4]:
def publish_observations(posex,x_mat_flatx,ft_valuesx):
    '''Function that computes and publishes the observations given the pose, rotation matrix and force torque data'''
    pos = posex.copy()[:3]
    rpy = posex.copy()[3:]
    x_mat = x_mat_flatx.copy().reshape((3,3))
    obs = numpy.concatenate([
        x_mat.dot(pos), x_mat.dot(normalize_rad(rpy)), ft_valuesx.copy()
    ])
    observation_publisher.publish(obs)

In [5]:
rospy.init_node("observator", anonymous=True)
observation_publisher = rospy.Publisher("/observation0", CustomVector, queue_size=1)
rospy.Subscriber("/rotation_matrix", CustomVector, rotmat_callback)
rospy.Subscriber("/ft300_force_torque", WrenchStamped, ft_callback)
rospy.Subscriber("/pose_estimation", EstimatedPose, pose_callback)
rate = rospy.Rate(1)

In [ ]:
while not rospy.is_shutdown():
    publish_observations(pose, x_mat_flat, ft_values)
    rate.sleep()

[WARN] [1593004021.234314]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [8]:
pose

array([-0.01488004,  0.02794781,  0.18030093, -3.12095371, -0.04345509,
        0.08726412])

In [11]:
x_mat_flat.reshape((3,3))

array([[-1.00000000e+00, -4.59035946e-05,  2.27224632e-06],
       [ 2.27388000e-06, -3.55858538e-05,  1.00000000e+00],
       [-4.59035109e-05,  1.00000000e+00,  3.55859593e-05]])

In [12]:
ft_values

array([-2.8300000e+00, -1.9200000e+00, -3.7290001e+01, -3.0000000e-02,
       -1.3000000e-02,  3.2000000e-01])

In [ ]:
def print(pose):
    print(pose)

In [ ]:
print(pose)